In [27]:
import requests
import json
baseurl = "http://localhost:3000/"

In [48]:
users = []
toks = {}
with open("users.json") as u:
    users = json.loads(u.read())
for u in users:
    payload = {'username':u['username'],'password':u['password'],'appcode':u['appcode']}
    res = requests.post(f"{baseurl}auth/login/",json=payload)
    _t = json.loads(res.text)
    toks[u['username']] = _t["token"]

In [50]:
for u in users:
    headers = {
        "authorization":f"bearer {toks[u['username']]}"
    }
    #print(headers)
    payload = {
    "app_id": u["appcode"],
    "loyalty_pid": u["valid_lp"][1],
    "user_id": f"{u["username"]}_id",
    "member_id": f"{u["username"]}_m_id",
    "member_first": f"{u["member_first"]}",
    "member_last":f"{u["member_last"]}",
    "transaction_date": "20240808",
    "ref_num": f"{abs(hash(u["username"]))}",
    "amount": 10000,
    "additional_info": "tonnes of additionl info"
    }
    res = requests.post(f"{baseurl}transact/add_record",json=payload,headers = headers)
    _t = json.loads(res.text)
    if(res.status_code!=200):
        print(f"{u["username"]} payload transact fail - {res.status_code}")

In [51]:
import glob
listing = glob.glob('../AccrualFiles/*')
for l in listing:
    hbs = "transfer_date,amount,reference_number,outcome_code\n" 
    with open(l) as f:
        for ll in (f.read().split('\n'))[1::]:
            r = ll.split(',')
            if(len(r)>1):
                hbs+=f"{r[4]},{r[5]},{r[6]},0000\n"

    with open(f"../HandBackFiles/{l[16::].replace("ACCRUAL","HANDBACK")}",'w+') as ff:
        ff.write(hbs)
